In [ ]:
!pip install pmdarima

In [1]:
from pmdarima.arima import auto_arima,OCSBTest,KPSSTest,ADFTest
from tqdm import tqdm
from sklearn import metrics
import pandas as pd
import numpy as np
import os
# from stationarizer import simple_auto_stationarize
%matplotlib inline

C:\Users\Hp\AppData\Roaming\Python\Python37\site-packages\statsmodels\tools\_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [101]:
calendar = pd.read_csv("calendar.csv")
sales_data = pd.read_csv("sales_train_validation.csv")
eval_data = pd.read_csv("sales_train_evaluation.csv")
sell_price = pd.read_csv("sell_prices.csv")
submissions = pd.read_csv("sample_submission.csv")

In [88]:
eval_data.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1549,d_1550,d_1551,d_1552,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,CA_1_HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,0,1,0,0,...,2,4,0,0,0,0,3,3,0,1
1,CA_1_HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,0,1,0,0,...,0,1,2,1,1,0,0,0,0,0
2,CA_1_HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,CA_1_HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,3,0,2,1,...,1,1,0,4,0,1,3,0,2,6
4,CA_1_HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,0,2,1,2,...,0,0,0,2,1,0,0,2,1,0
5,CA_1_HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_006,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,1,1,3,1,...,2,1,0,0,1,0,0,5,2,0
6,CA_1_HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0
7,CA_1_HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,11,7,30,4,...,7,0,6,0,15,5,4,1,40,32
8,CA_1_HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
9,CA_1_HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,CA_1_HOBBIES_1,CA_1_HOBBIES,CA_1,CA,2,1,0,0,...,0,0,1,0,2,1,1,0,0,1


In [102]:
eval_data['cat_id']  = (eval_data['store_id'] + '_' + eval_data['cat_id'])
eval_data['dept_id'] = (eval_data['store_id'] + '_' + eval_data['dept_id'])
eval_data['id'] = (eval_data['store_id'] + '_' + eval_data['id'])
days = ['d_'+str(i) for i in range(1,1549)]
eval_data = eval_data.drop(columns=days)

In [103]:
hierarchy = {0: "state_id", 1: "store_id", 2: "cat_id", 3: "dept_id", 4: "id"}

In [104]:
## make heirarchical dataframe 
def generate_heirarchy(eval_data):
    final_work = pd.DataFrame()
    for i in hierarchy:
      col = hierarchy[i]
      df_1 = eval_data.groupby(by = [col],as_index= False).sum()
      data_final = df_1.T
      data_final.columns = data_final.loc[col]
      data_final=data_final.drop(col)
      if final_work.empty:
        final_work = data_final.copy()
      else:
        final_work = pd.merge(final_work,data_final,left_index=True, right_index=True)
    final_work['total'] = final_work['CA']+final_work['TX']+final_work['WI']
    final_work= final_work.T
    final_work.to_csv("final_work.csv")
    return final_work

In [105]:
# remove trends and/or seasonality from the data using OSCB test

def stationarize(one, counter=0):    
    try:
        trend_result = ADFTest().should_diff(one)[1]  
        if trend_result and counter<2:
            one = one.diff()
            one =one.dropna()
            counter += 1
            one = stationarize(one, counter)
    except:
        None
    return one

def season_stationary(one):
    try:
        season_result = OCSBTest(m=30).estimate_seasonal_differencing_term(one)
        if season_result:
            one = one.diff(30)
            one =one.dropna()
    except:
        None
    return one
  


In [92]:
# make predictions for the entire heirarchy on each level

# I call this function on different ranges on different machines at the same time  

def make_predictions(final_work, start=0, end=0):
    if end==0:
        end = final_work.shape[0]
    for i in tqdm(range(start,end), position=0, leave=True):
        one = final_work.iloc[i]
        id = final_work.index[i]
        # one = one.drop(['id'])
        one = stationarize(one)
        one = season_stationary(one)
        model_fit = auto_arima(one, m=1, suppress_warnings = True, error_action="ignore")
        pred = model_fit.predict(28)
        result_sales.loc[id] = pred

    result_sales.dropna()
    num = result_sales._get_numeric_data()
    num[num < 0] = 0
    result_sales.to_csv(f'eval_{start}.csv')

    return result_sales


In [95]:
final_work = generate_heirarchy(sales_data)
pred_days= ['F'+ str(i) for i in range(1,29)]
result_sales = pd.DataFrame(columns=pred_days, index = final_work.index)
make_predictions(final_work)

  0%|          | 0/30514 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [96]:
total_pred = pd.read_csv('eval_final.csv')

In [97]:
states = eval_data.state_id.unique()
stores = eval_data.store_id.unique()
depts = eval_data.dept_id.unique()
cats = eval_data.cat_id.unique()
items = eval_data.id.unique()

# Here we build the tree as a dictionary. Each node (key in dict) has a list of 
# children value in dict, which in turn may also be a key in the dict, and have 
# children as well 

total = {'total': list(states)}
state_h = {k: [v for v in stores if v.startswith(k)] for k in states}
store_h = {k: [v for v in cats if v.startswith(k)] for k in stores}
dept_h = {k: [v for v in depts if v.startswith(k)] for k in cats}
item_h = {k: [v for v in items if v.startswith(k)] for k in depts}

In [98]:
hier_dict = {**total, **state_h, **store_h, **dept_h, **item_h}
hier_total = list(states)+list(stores)+list(cats)+list(depts)
hier_total.insert(0,'total')

In [99]:
# total_pred.index = total_pred['Unnamed: 0']
total_pred = total_pred.rename(columns={'Unnamed: 0':'id'})

total_pred = total_pred[~total_pred['id'].isin(hier_total)]

In [100]:
hr = eval_data.iloc[:,0:6]
heir_pred_valid = pd.merge(hr,total_pred,on = 'id')

In [50]:
## rework the entire hierarchy after prediction 

pred_final = generate_heirarchy(heir_pred_valid)

In [51]:
pred_final.to_csv('pred_all_sums.csv')

In [52]:
all_predi = pd.read_csv('eval_final.csv')
all_sum = pd.read_csv('pred_all_sums.csv')

In [53]:
num = all_predi._get_numeric_data()
num[num < 0] = 0

In [55]:
# final_work = final_work.T
all_sum.head()

,Unnamed: 0,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,CA,15090.757440,15109.921754,15092.817622,15370.550563,15701.243985,15925.522647,15813.174370,15433.347177,15172.285190,...,15655.799846,15898.122188,15760.673984,15464.219986,15204.649232,15180.319114,15379.265271,15670.419010,15877.240973,15785.812598
1,TX,9462.561603,9588.252172,9692.426644,9810.208549,9940.528557,9945.319140,9879.969903,9711.944512,9639.912921,...,9883.362007,9898.144092,9813.946979,9678.493940,9648.075776,9663.930163,9775.400506,9861.940095,9890.916928,9801.926060
2,WI,11295.696181,11396.032347,11464.214064,11585.396416,11756.980441,11838.258856,11730.203993,11599.796796,11527.803722,...,11849.068460,11940.984657,11853.728186,11709.439616,11624.493118,11652.462224,11731.447888,11876.932465,11947.125980,11869.217164
3,CA_1,3984.110533,3905.465823,3892.948589,3978.226746,4110.349611,4211.885134,4186.752599,4046.041331,3933.794980,...,4076.113692,4194.525566,4147.315970,4038.075791,3931.745407,3896.647980,3964.457007,4077.642518,4175.767682,4150.250619
4,CA_2,3808.506878,3779.947126,3824.146178,3941.503069,4059.369683,4119.164553,4032.540977,3889.121915,3817.755229,...,4082.164721,4126.384992,4050.090348,3932.318824,3861.865838,3890.196022,3987.913131,4090.917949,4139.075679,4076.057313


In [56]:
all_predi.index = all_predi['Unnamed: 0']
all_sum.index = all_sum['Unnamed: 0']

all_predi = all_predi.drop(columns = ['Unnamed: 0'])
all_sum = all_sum.drop(columns = ['Unnamed: 0'])

all_predi = all_predi.T
all_sum = all_sum.T


In [ ]:
# hier_pred = [a for a in final_work.columns if a.endswith('validation')]

In [57]:
# getting disaggregation proportions as mentioned by Hyndman

## p = Y/S 
# Y = predictions on that level
# S = sum of predictions of all levels below that level

def get_proportions(Y, S):
  Y_arr = Y.to_numpy(dtype='float32')
  S_arr = S.to_numpy(dtype='float32')
  P_arr = np.true_divide(Y_arr,S_arr, out=np.zeros_like(Y_arr), where=S_arr!=0)
  P = pd.Series(P_arr)
  return P

In [58]:
extremely_final = pd.DataFrame()
extremely_final['total'] = all_predi['total'] 

In [59]:
# Now we disaggregate the proportions to each level

for i in hier_total:
  p = get_proportions(extremely_final[i],all_sum[i])
  for j in hier_dict[i]:
    x = all_predi[j].to_numpy(dtype='float32')
    y = p.to_numpy(dtype='float32')
    m = np.multiply(x,y)
    extremely_final[j] = m.tolist()

In [61]:
extremely_final = extremely_final.drop(columns=hier_total)
extremely_final = extremely_final.T

In [70]:
extremely_final['id'] = extremely_final.index
extremely_final['id'] = extremely_final['id'].str[5:]

AttributeError: 'DataFrame' object has no attribute 'str'

In [65]:
extremely_final.to_csv('extremely_final.csv')